In [1]:
import pandas as pd
import numpy as np
import joblib
import sklearn
from catboost import CatBoostRegressor, Pool

In [2]:
import re
from tqdm.notebook import tqdm
tqdm.pandas()

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier

In [3]:
from nltk import sent_tokenize, word_tokenize, regexp_tokenize
#nltk.download('stopwords') 
import nltk
import spacy

stopwords_nltk = nltk.corpus.stopwords.words('russian') #лист русский стоп-слов 
stopwords_nltk_en = nltk.corpus.stopwords.words('english') 
stopwords_nltk.extend(stopwords_nltk_en) #если есть текста на английском

new_stop = []
stopwords_nltk.extend(new_stop) 

lemmatizer = spacy.load('ru_core_news_md', disable = ['parser', 'ner'])

In [4]:
print(nltk.__version__)
print(spacy.__version__)

3.7
3.7.2


In [5]:
#очистка текста
def full_clean(text):
    '''подготовка текста к подаче в модель для каждого текста (через applay)'''
    try:
        text=re.sub(r"[^a-zA-Zа-яА-ЯёЁ0-9#]", " ", text)
        text = text.lower()
        text = re.sub(" +", " ", text).strip() #оставляем только 1 пробел
    except:
        text = ' '
    #токены для моделей
    tokens = [token.lemma_ for token in lemmatizer(text) if token.lemma_ not in stopwords_nltk]
    #для tfidf на вход текст
    text = " ".join(tokens)
    return text, tokens


def preprocess_text(df):
    '''подготовка текста к подаче в модель колонкой'''
    new_corpus=[]
    new_tokens=[]

    for text in tqdm(df):
        text, tokens = full_clean(text)
        new_corpus.append(text)
        new_tokens.append(tokens)
        
    return new_corpus, new_tokens

In [6]:
def tfidf_embeding(model=None, df=None):   
    '''Преобразование текста в мешок слов'''
    if model==None:
        #загрузить если нет
        model = joblib.load('tfidf.pkl')
    else:
        model=model
    X = model.transform(df)
    #feature_names = model.get_feature_names_out()
    #X = pd.DataFrame(model.toarray(), columns = feature_names, index = df.index)
    
    return X.toarray()#.tolist()

In [7]:
def create_submission_part(test_df, name_of_predict_column, value):
    submission = pd.DataFrame([])
    submission['id'] = test_df['id']
    submission[name_of_predict_column] = value
    return submission

def create_submission(RES_part, SAL_part):
    RES_part['task_type'] = 'RES'
    SAL_part['task_type'] = 'SAL'
    submission = pd.concat([RES_part, SAL_part], axis=0)
    return submission

Предсказание профессии

In [9]:
df_test_RES = pd.read_csv('vprod_test/TEST_RES.csv') #nrows=4000 для примера
df_test_RES

,achievements,achievements_modified,company_name,demands,id
0,NaN,NaN,NaN,Уборка территории,445ad275-7a83-11ec-9ed5-57fc951f3846
1,NaN,NaN,NaN,Выполняет работы по очистке территорий от сухо...,444f5e15-f88e-11ed-9b14-05dc90903fb8
2,NaN,NaN,NaN,- проведение своевременной уборки групповой ко...,4448c865-1afe-11ee-9f4d-9586bb63c653
3,NaN,NaN,NaN,Погрузка товара,444601b5-0ea7-11ee-9332-9586bb63c653
4,NaN,NaN,NaN,Погрузка разгрузка,4445e898-140c-11ef-a0f7-d549be31d974
...,...,...,...,...,...
1951,NaN,NaN,NaN,Преподавание английского языка (начальные клас...,b53b7800-1fe5-11ef-b229-d7f294e00a65
1952,NaN,NaN,NaN,Осуществляет оперативное руководство единой см...,b533bda5-6f05-11ed-b6c5-91f5d381286a
1953,NaN,NaN,NaN,ведение и лечение неврологических пациентов,b5313075-be34-11ed-b9c1-05dc90903fb8
1954,NaN,NaN,NaN,Заведующий складским хозяйством. Работа на скл...,b50cd4f6-d549-11ee-9f2d-e7d0d2cf29b1


In [10]:
tfidf = joblib.load('tfidf.pkl')
logit = joblib.load('logit.pkl')

In [11]:
#df_test_RES = df_test_RES.rename(columns = {"id_cv" : "id"})

In [12]:
#только для примера
#df_test_RES = df_test_RES[["id", 'demands']].sample(n=4000, random_state=1)

In [13]:
df_test_RES['text_clean'], df_test_RES['tokens']=preprocess_text(df_test_RES['demands'])

  0%|          | 0/1956 [00:00<?, ?it/s]

In [14]:
tfidf_test = tfidf_embeding(model=tfidf, df=df_test_RES['text_clean'])

In [15]:
model_test = logit.predict(tfidf_test)

In [16]:
submission_RES_part = create_submission_part(df_test_RES, 'job_title', model_test)

In [17]:
#submission_RES_part.to_csv('submission_RES_part.csv', index = False)

Предсказание зарплаты

In [18]:
df_TEST_SAL = pd.read_csv('vprod_test/TEST_SAL.csv') #nrows=4000 для примера
df_TEST_SAL

,id,academic_degree,accommodation_capability,accommodation_type,additional_premium,additional_requirements,bonus_type,measure_type,busy_type,career_perspective,...,federalDistrictCode,industryBranchName,contactList,company_name,full_company_name,company_inn,company,languageKnowledge,hardSkills,softSkills
0,f47e56d8-391d-11ef-8f5c-e7d0d2cf29b1,NaN,False,NaN,NaN,NaN,NaN,NaN,Полная занятость,False,...,1.0,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ПРАК...",7.718704e+09,"{""companycode"":""1087746619487"",""email"":""moe@pr...",[],"[{""hard_skill_name"":""контроль исполнения докум...","[{""owner_id"":""f47e56d8-391d-11ef-8f5c-e7d0d2cf..."
1,f90390db-391d-11ef-974d-d549be31d974,NaN,False,NaN,NaN,NaN,NaN,NaN,Полная занятость,False,...,1.0,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ТРАК...",7.743898e+09,"{""companycode"":""1137746734399"",""email"":""Pchelk...",[],[],[]
2,f8fcb308-391d-11ef-b4d9-cb26dff57dd7,NaN,False,NaN,NaN,NaN,NaN,NaN,Частичная занятость,False,...,3.0,NaN,NaN,NaN,"муниципальное бюджетное учреждение ""Спортивная...",2.360000e+09,"{""companycode"":""1082360000106"",""email"":""Legion...",[],[],[]
3,f936fce8-391d-11ef-b4d9-cb26dff57dd7,NaN,False,NaN,10000.0,NaN,QUARTERLY,RUBLE,Полная занятость,True,...,6.0,NaN,NaN,NaN,"ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""КОЛОНИЯ-ПОСЕЛ...",4.244001e+09,"{""companycode"":""1024201368740"",""email"":""kp3@42...",[],[],[]
4,fb1580e3-391d-11ef-ab54-ef6fdced905e,NaN,False,NaN,NaN,NaN,NaN,NaN,Полная занятость,False,...,1.0,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ТРИМ...",3.235016e+09,"{""companycode"":""1033231004223"",""email"":""trimst...",[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1951,dfa744f8-3946-11ef-8f5c-e7d0d2cf29b1,NaN,False,NaN,50000.0,<ul> <li>опыт работы не обязателен.</li> </ul>,ANNUAL,RUBLE,Полная занятость,False,...,4.0,NaN,NaN,NaN,РУЗАЕВСКИЙ ЛИНЕЙНЫЙ ОТДЕЛ МИНИСТЕРСТВА ВНУТРЕН...,1.324125e+09,"{""companycode"":""1021300928703"",""email"":""lovd95...",[],[],[]
1952,e055ad68-3946-11ef-8f5c-e7d0d2cf29b1,NaN,True,NaN,NaN,NaN,NaN,NaN,Полная занятость,False,...,1.0,NaN,NaN,NaN,МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ДОПОЛНИТЕЛЬ...,6.911014e+09,"{""companycode"":""1026901734265"",""hr-agency"":fal...",[],[],[]
1953,50799028-3947-11ef-8f5c-e7d0d2cf29b1,NaN,False,NaN,NaN,NaN,NaN,NaN,Полная занятость,False,...,8.0,NaN,NaN,NaN,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,2.628004e+09,"{""companycode"":""1022601320631"",""email"":""kspoli...",[],[],[]
1954,7b837a18-3947-11ef-8f5c-e7d0d2cf29b1,NaN,False,NaN,NaN,"<ul> <li>оформление в соответствии с ТК РФ, по...",NaN,NaN,Полная занятость,False,...,2.0,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ОТЕЛ...",7.839132e+09,"{""companycode"":""1207800127875"",""email"":""info@k...",[],"[{""hard_skill_name"":""вежливость"",""owner_id"":""7...",[]


In [19]:
model = CatBoostRegressor(learning_rate=0.1,iterations=10000,max_depth = 6)

In [20]:
model.load_model('model_1.cbm')

In [21]:
col = [ 'busy_type','education', 'education_speciality', 'regionName', 'company_business_size', 
       'required_experience',  'schedule_type', 'source_type', 'professionalSphereName', 'federalDistrictCode']

cat_features = ['busy_type','education', 'education_speciality', 'regionName', 'company_business_size', 
                'schedule_type', 'source_type', 'professionalSphereName']

In [22]:
X_test=df_TEST_SAL[col]
X_test=X_test.fillna(0)
#Y_test=0

In [23]:
test_pool = Pool(X_test, cat_features=cat_features)

In [24]:
test_predict = model.predict(test_pool)

In [25]:
submission_SAL_part = create_submission_part(df_TEST_SAL, 'salary', test_predict.round(1))

In [26]:
submission = create_submission(submission_RES_part, submission_SAL_part)

In [27]:
submission

,id,job_title,task_type,salary
0,445ad275-7a83-11ec-9ed5-57fc951f3846,дворник,RES,NaN
1,444f5e15-f88e-11ed-9b14-05dc90903fb8,рабочий зеленого хозяйства,RES,NaN
2,4448c865-1afe-11ee-9f4d-9586bb63c653,помощник воспитателя,RES,NaN
3,444601b5-0ea7-11ee-9332-9586bb63c653,грузчик,RES,NaN
4,4445e898-140c-11ef-a0f7-d549be31d974,грузчик,RES,NaN
...,...,...,...,...
1951,dfa744f8-3946-11ef-8f5c-e7d0d2cf29b1,NaN,SAL,23226.6
1952,e055ad68-3946-11ef-8f5c-e7d0d2cf29b1,NaN,SAL,23635.8
1953,50799028-3947-11ef-8f5c-e7d0d2cf29b1,NaN,SAL,21086.3
1954,7b837a18-3947-11ef-8f5c-e7d0d2cf29b1,NaN,SAL,47070.3


In [28]:
submission.to_csv('submission.csv', index = False)